# 问题
你正在处理Unicode字符串，需要确保所有字符串在底层有相同的表示。

# 解决方案
在Unicode中，某些字符能够用多个合法的编码表示。为了说明，考虑下面的这个例子：

In [1]:
s1 = 'Spicy Jalape\u00f1o'
s2 = 'Spicy Jalapen\u0303o'
s1

'Spicy Jalapeño'

In [2]:
s2

'Spicy Jalapeño'

In [3]:
s1 == s2

False

In [4]:
len(s1)

14

In [5]:
len(s2)

15

这里的文本”Spicy Jalapeño”使用了两种形式来表示。 第一种使用整体字符”ñ”(U+00F1)，第二种使用拉丁字母”n”后面跟一个”~”的组合字符(U+0303)。

在需要比较字符串的程序中使用字符的多种表示会产生问题。 为了修正这个问题，你可以使用unicodedata模块先将文本标准化：

In [6]:
import unicodedata
t1 = unicodedata.normalize('NFC', s1)
t2 = unicodedata.normalize('NFC', s2)
print(t1 == t2)

print(ascii(t1))

t3 = unicodedata.normalize('NFD', s1)
t4 = unicodedata.normalize('NFD', s2)
print(t3 == t4)

print(ascii(t3))


True
'Spicy Jalape\xf1o'
True
'Spicy Jalapen\u0303o'


normalize() 第一个参数指定字符串标准化的方式。 NFC表示字符应该是整体组成(比如可能的话就使用单一编码)，而NFD表示字符应该分解为多个组合字符表示。

Python同样支持扩展的标准化形式NFKC和NFKD，它们在处理某些字符的时候增加了额外的兼容特性。比如：

In [7]:
s = '\ufb01' # A single character
print(s)

ﬁ


In [8]:
unicodedata.normalize('NFD', s)

'ﬁ'

In [9]:
unicodedata.normalize('NFKD', s)

'fi'

In [10]:
unicodedata.normalize('NFKC', s)

'fi'

# 写成一个函数

In [12]:
import unicodedata
import string

all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)
# Turn a Unicode string to plain ASCII, thanks to https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

print(unicodeToAscii('Ślusàrski'))

Slusarski
